# Classifying comments in r/BlackPeopleTwitter and r/WhitePeopleTwitter

With 1,726,152 subscribers, BlackPeopleTwitter is one of the most popular subreddits on Reddit. In the words of the sub's description:  
Screenshots of Black people being hilarious and insightful on social media, it doesn't need to just be twitter but obviously that is best.  
Black culture has a unique way of examining the everyday and we are here to showcase that.  
A community for three years.

With 515,361 subscribers and also founded three years ago, WhitePeopleTwitter doesn't have a description but seems to be similar to BlackPeopleTwitter. The existence of two popular subreddits that are similar but organized around a different putative racial category is interesting.  

The posts are screenshots from Twitter, and are either posted directly to Reddit in various image formats (e.g. .jpg) or are hosted on Imgur, an image hosting site. This means that we can't use natural language processing directly on the posts. Instead, we'll be looking at the comments. Do Reddit users use different enough language when responding to 'black' and 'white' Twitter posts that we can build a classifier?

Additional problem: looking at comments means that we won't have data for posts that have no comments. It's fair to throw this data away since the question we've defined is about reactions.

In [5]:
import pandas as pd
import requests
import json
import time

url_base = "http://www.reddit.com/"

slug_hot = "hot.json"

slug_bpt = "r/BlackPeopleTwitter/" # an optional intermediate slug to throw in to view a specific subreddit's 'hot' page
slug_wpt = "r/WhitePeopleTwitter/"

user = {'User-agent': 'JonBot 0.1'} # I need a User-agent to get in

---
Starting with BlackPeopleTwitter:

In [2]:
url_bpt = url_base + slug_bpt + slug_hot
res = requests.get(url_bpt, headers = user)
data = res.json()
i = 0 # this is the post number on this page
slug_bpt_id = data['data']['children'][i]['data']['id']
comments_this_post = data['data']['children'][i]['data']['num_comments'] # this is a requirement for the project

In [3]:
url_bpt_comments = url_base + slug_bpt + 'comments/' + slug_bpt_id + '.json'

res = requests.get(url_bpt_comments, headers = user)

res.status_code

200

In [4]:
url_bpt_comments

'http://www.reddit.com/r/BlackPeopleTwitter/comments/8mcaq9.json'

In [5]:
data = res.json()

In [6]:
data[0]

{'data': {'after': None,
  'before': None,
  'children': [{'data': {'approved_at_utc': None,
     'approved_by': None,
     'archived': False,
     'author': 'Nathan561',
     'author_flair_css_class': 'mod',
     'author_flair_template_id': None,
     'author_flair_text': 'Mod',
     'banned_at_utc': None,
     'banned_by': None,
     'can_gild': False,
     'can_mod_post': False,
     'clicked': False,
     'contest_mode': False,
     'created': 1527390320.0,
     'created_utc': 1527361520.0,
     'distinguished': 'moderator',
     'domain': 'self.BlackPeopleTwitter',
     'downs': 0,
     'edited': False,
     'gilded': 0,
     'hidden': False,
     'hide_score': False,
     'id': '8mcaq9',
     'is_crosspostable': False,
     'is_reddit_media_domain': False,
     'is_self': True,
     'is_video': False,
     'likes': None,
     'link_flair_css_class': None,
     'link_flair_text': None,
     'locked': False,
     'media': None,
     'media_embed': {},
     'media_only': False,
    

In [7]:
for j in range(5):
    print(data[1]['data']['children'][j]['data']['body']) # Here's how to cycle through the comments for one post

Fuck I thought it was Sophie Dee. That would have been the only way I could have forgiven drake. 

In other news I think I’m gonna delete this account and take a long ass break from Reddit. Maybe I’ll lurk. I really wish the site were just memes and sports, but the toxic shit is like a magnet to me and I can’t let it ruin me. 
graduating high school in 3 days and my average isn't enough for university. might have to stick around this shit hole for another year. how was your day?
Men get enough perks.  Can't women just have something nice for themselves?
Tobe Nwigwe is 🔥
I'm trying to off my ugly ass self 


In [8]:
list(data[1]['data']['children'][0]['data'].keys()) # Here's all the keys for each comment

['subreddit_id',
 'approved_at_utc',
 'ups',
 'mod_reason_by',
 'banned_by',
 'removal_reason',
 'link_id',
 'author_flair_template_id',
 'likes',
 'no_follow',
 'replies',
 'user_reports',
 'saved',
 'id',
 'banned_at_utc',
 'mod_reason_title',
 'gilded',
 'archived',
 'report_reasons',
 'author',
 'can_mod_post',
 'send_replies',
 'parent_id',
 'score',
 'approved_by',
 'downs',
 'body',
 'edited',
 'author_flair_css_class',
 'collapsed',
 'is_submitter',
 'collapsed_reason',
 'body_html',
 'subreddit_type',
 'can_gild',
 'subreddit',
 'name',
 'score_hidden',
 'permalink',
 'num_reports',
 'stickied',
 'created',
 'author_flair_text',
 'created_utc',
 'subreddit_name_prefixed',
 'controversiality',
 'depth',
 'mod_reports',
 'mod_note',
 'distinguished']

By looking through the keys, I'm going to select the features that are most likely to be useful. At minimum, I'm going to pull out `body`, `subreddit_name_prefixed`, `created_utc`, and `body`.

In [9]:
features = ['created_utc','subreddit_name_prefixed', 'body', 'ups']
for feature in features:
    print(feature + ":", data[1]['data']['children'][0]['data'][feature])

created_utc: 1527656802.0
subreddit_name_prefixed: r/BlackPeopleTwitter
body: Fuck I thought it was Sophie Dee. That would have been the only way I could have forgiven drake. 

In other news I think I’m gonna delete this account and take a long ass break from Reddit. Maybe I’ll lurk. I really wish the site were just memes and sports, but the toxic shit is like a magnet to me and I can’t let it ruin me. 
ups: 5


Let's try to loop these.

In [10]:
url_bpt = url_base + slug_bpt + slug_hot + '?after=t3_8mwi64'
url_bpt

'http://www.reddit.com/r/BlackPeopleTwitter/hot.json?after=t3_8mwi64'

In [11]:
res = requests.get(url_bpt, headers = user)

In [12]:
data = res.json()

In [13]:
data['data']['children'][0]['data']['selftext']

''

In [14]:
import time
list_of_dictionaries = []
for i in range(25):
    
# Setting up the top 
    this_dict = {}
    url_bpt = url_base + slug_bpt + slug_hot
    res = requests.get(url_bpt, headers = user)
    data = res.json()
    
    slug_bpt_id = data['data']['children'][i]['data']['id']
    comments_this_post = data['data']['children'][i]['data']['num_comments']
    post_title = data['data']['children'][i]['data']['title']
    
    if (i+1)%25==0:
        print(data['data']['after'])
    
    this_dict['comments_this_post'] = comments_this_post
    this_dict['post_title'] = post_title
    
    url_bpt_comments = url_base + slug_bpt + 'comments/' + slug_bpt_id + '.json'
    res = requests.get(url_bpt_comments, headers = user)
    data = res.json()
    features = ['created_utc','subreddit_name_prefixed', 'body', 'ups']
    for feature in features:
        this_dict[feature] = data[1]['data']['children'][0]['data'][feature]
    time.sleep(1.1)
    list_of_dictionaries.append(this_dict)

pd.DataFrame(list_of_dictionaries)

t3_8n2a97


,body,comments_this_post,created_utc,post_title,subreddit_name_prefixed,ups
0,Fuck I thought it was Sophie Dee. That would h...,138,1.527657e+09,/r/BlackPeopleTwitter Weekly Discussion Thread...,r/BlackPeopleTwitter,4
1,"As a surprise to zero people, Pusha T is a lot...",590,1.527642e+09,Pusha-T Responds to Drake,r/BlackPeopleTwitter,1466
2,"That or ""I am hiding a child, but you're hidin...",68,1.527688e+09,King Push vs King Pushover,r/BlackPeopleTwitter,421
3,She should have talked about Mexicans. Nobody ...,1151,1.527651e+09,You can understand how she'd be confused,r/BlackPeopleTwitter,3269
4,I’ve never seen a single tweet blow up in a ce...,66,1.527688e+09,When the Ambien hits,r/BlackPeopleTwitter,56
5,Boi just got \n#Lit Up,27,1.527686e+09,UR BABY MAMA STINK,r/BlackPeopleTwitter,137
6,I didn't know he had kids,110,1.527646e+09,"Only partly, son",r/BlackPeopleTwitter,605
7,"Meek Mill is in the studio right now, flipping...",93,1.527650e+09,Pulitzer T,r/BlackPeopleTwitter,1378
8,Gonna start calling Push uncle and it’s gonna ...,11,1.527684e+09,Better that than not spending any time with hi...,r/BlackPeopleTwitter,56
9,That was one of the least rewarding shows I've...,25,1.527690e+09,A true life achievement,r/BlackPeopleTwitter,79


In [15]:
# import time
# list_of_dictionaries = []
# aft = ''
# features = ['created_utc', 'body', 'ups']
# for i in range(100):
#     j = i % 25
#     this_dict = {}
#     url_bpt = url_base + slug_bpt + slug_hot + aft
#     res = requests.get(url_bpt, headers = user)
#     data = res.json()
    
#     slug_bpt_id = data['data']['children'][j]['data']['id']
#     comments_this_post = data['data']['children'][j]['data']['num_comments']
#     post_title = data['data']['children'][j]['data']['title']
    
#     if (i+1)%25==0:
#         print(data['data']['after'])
#         aft = '?after='+data['data']['after']
    
#     this_dict['comments_this_post'] = comments_this_post
#     this_dict['post_title'] = post_title
# #     print(post_title)
#     url_bpt_comments = url_base + slug_bpt + 'comments/' + slug_bpt_id + '.json'
#     res = requests.get(url_bpt_comments, headers = user)
#     data = res.json()
#     try:
#         comment_data = data[1]['data']['children'][0]['data']
#     except:
#         pass
#     for feature in features:
#         this_dict[feature] = comment_data[feature]
# #     print(comment_data['ups'])
#     time.sleep(1.1)
#     list_of_dictionaries.append(this_dict)

# pd.DataFrame(list_of_dictionaries)

In [ ]:
# df_bpt100 = pd.DataFrame(list_of_dictionaries)
# df_bpt100['sub'] = 0

# df_bpt100.head()

In [18]:
list_of_dictionaries = []
aft = ''
features = ['created_utc', 'body', 'ups']

for i in range(2000):
    j = i % 25
    this_dict = {}
    url_bpt = url_base + slug_bpt + slug_hot + aft
    res = requests.get(url_bpt, headers = user)
    data = res.json()

    slug_bpt_id = data['data']['children'][j]['data']['id']
    comments_this_post = data['data']['children'][j]['data']['num_comments']
    post_title = data['data']['children'][j]['data']['title']
    
    if (i+1)%25==0:
        print("{} posts scraped!".format(i+1))
        aft = '?after='+data['data']['after']
    
    this_dict['comments_this_post'] = comments_this_post
    this_dict['post_title'] = post_title
#     print(post_title)
    url_bpt_comments = url_base + slug_bpt + 'comments/' + slug_bpt_id + '.json'
    res = requests.get(url_bpt_comments, headers = user)
    data = res.json()
    try:
        comment_data = data[1]['data']['children'][0]['data']
    except:
        pass
    for feature in features:
        this_dict[feature] = comment_data[feature]
#     print(comment_data['ups'])
    time.sleep(3)
    list_of_dictionaries.append(this_dict)

pd.DataFrame(list_of_dictionaries).to_csv('Bpt_2000.csv')

25 posts scraped!
50 posts scraped!
75 posts scraped!
100 posts scraped!
125 posts scraped!
150 posts scraped!
175 posts scraped!
200 posts scraped!
225 posts scraped!
250 posts scraped!
275 posts scraped!
300 posts scraped!
325 posts scraped!
350 posts scraped!
375 posts scraped!
400 posts scraped!
425 posts scraped!
450 posts scraped!
475 posts scraped!
500 posts scraped!
525 posts scraped!
550 posts scraped!
575 posts scraped!
600 posts scraped!
625 posts scraped!
650 posts scraped!
675 posts scraped!
700 posts scraped!
725 posts scraped!
750 posts scraped!
775 posts scraped!
800 posts scraped!
825 posts scraped!


IndexError: list index out of range

In [21]:
pd.DataFrame(list_of_dictionaries).to_csv("bpt_scraped_109pm_may30.csv")

In [33]:
slug_wpt = 'r/WhitePeopleTwitter/'

In [34]:
slug_hot

'hot.json'

In [6]:
list_of_dictionaries = []
aft = ''
features = ['created_utc', 'body', 'ups']

for i in range(1000):
    j = i % 25
    this_dict = {}
    url_wpt = url_base + slug_wpt + slug_hot + aft
    res = requests.get(url_wpt, headers = user)
    data = res.json()

    slug_wpt_id = data['data']['children'][j]['data']['id']
    comments_this_post = data['data']['children'][j]['data']['num_comments']
    post_title = data['data']['children'][j]['data']['title']
    
    if (i+1)%25==0:
        print("{} posts scraped!".format(i+1))
        aft = '?after='+data['data']['after']
    
    this_dict['comments_this_post'] = comments_this_post
    this_dict['post_title'] = post_title
#     print(post_title)
    url_wpt_comments = url_base + slug_wpt + 'comments/' + slug_wpt_id + '.json'
    res = requests.get(url_wpt_comments, headers = user)
    data = res.json()
    try:
        comment_data = data[1]['data']['children'][0]['data']
    except:
        pass
    for feature in features:
        this_dict[feature] = comment_data[feature]
#     print(comment_data['ups'])
    time.sleep(3)
    list_of_dictionaries.append(this_dict)

pd.DataFrame(list_of_dictionaries).to_csv('Wpt_1000.csv')

25 posts scraped!


KeyboardInterrupt: 

In [40]:
# 1527700578.5692391 for the bpt twitter thing from may 30.

# 1527704638.661053 for the wpt twitter thing from may 30.


In [2]:

list_of_dictionaries

NameError: name 'list_of_dictionaries' is not defined